In [2]:
from experiments._1_one_user_learn_neighbours.try_some_users import *
from experiments.utils import *
from experiments.datasets import *
import json

Switching to API Credentials #3


In [3]:
X_train, X_test, y_train, y_test = load_or_create_dataframe(uid)

In [4]:
s = open_session()

train_tweets = [s.query(Tweet).get(twid) for twid in X_train.index]

test_tweets = [s.query(Tweet).get(twid) for twid in X_test.index]

In [5]:
pos_train_tweets = [t for i, t in enumerate(train_tweets) if y_train[i]]

In [6]:
neg_train_tweets = [t for i, t in enumerate(train_tweets) if not y_train[i]]

In [34]:
[t.text for t in neg_train_tweets[:10]]

[u'Los jueces de Tucuman que frenaron las elecciones. Que sepan que les vamos a regalar pasajes a donde quieran #JulietaZylberbergEnTVR @TVRok',
 u'@orgullozombie Yo puedo decirlo. Vos no. Sorry. Es la pol\xedtica. No te pongas los ruleros. As\xed se dice,no?',
 u'@luisnovaresio http://t.co/3JB12ZXRHx',
 u'\u201cObviamente (la muerte del ni\xf1o qom) nos duele pero no podemos decir que el Estado no estuvo presente\u201d http://t.co/IV1yegjlCw',
 u'FernandezAnibal: A las medidas adoptadas por la provincia se suma AFIP, Banco Naci\xf3n y cr\xe9ditos del BID y la C... http://t.co/24dhMhRXoU',
 u'Memoria incompleta o solo bussines? https://t.co/EyuRftjafp',
 u'Video | Evacuaron el Luna Park por un principio de incendio http://t.co/0UOeUMETWN http://t.co/U8kX1rlapi',
 u'.@berlich .@nofumarx http://t.co/iYgqFhH3ae',
 u'CR\xd3NICA: tapa de la edici\xf3n impresa http://t.co/rrImQTCGmR',
 u'Capitanich reconoci\xf3 que en cuatro a\xf1os murieron 1.800 ni\xf1os a causa de la pobreza http://t.co/p

# Create Tf IDF model

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from tokenizer import tokenize, spanish_stopwords
from random import sample

In [ ]:
all_tweets_text_es = [t.text for t in s.query(Tweet).all() if t.lang == 'es']

In [18]:
vec = CountVectorizer(tokenizer=tokenize, min_df=30, max_df=0.5, stop_words=spanish_stopwords)

In [19]:
counts = vec.fit_transform(sample(all_tweets_text_es, 300000))

In [20]:
len(vec.vocabulary_)

8067

In [21]:
vec.vocabulary_

{u'igual': 3740,
 u'canel': 1158,
 u'dni': 2337,
 u'dinosauri': 2268,
 u'fir': 3095,
 u'fit': 3101,
 u'fib': 3060,
 u'fif': 3074,
 u'fij': 3077,
 u'fin': 3084,
 u'fil': 3079,
 u'unanim': 7485,
 u'conmemor': 1674,
 u'restaur': 6421,
 u'guille': 3466,
 u'\xbfquien': 7977,
 u'\xbfquier': 7978,
 u'despiert': 2161,
 u'frut': 3207,
 u'ballen': 752,
 u'exact': 2854,
 u'maltrat': 4595,
 u'httptcospywbwoeb': 3681,
 u'corazon': 1777,
 u'performanc': 5708,
 u'sistemat': 6888,
 u'petroler': 5753,
 u'aprob': 505,
 u'afp': 161,
 u'sangarciacorr': 6659,
 u'info': 3848,
 u'buenmiercol': 1017,
 u'incomod': 3799,
 u'seguidor': 6728,
 u'dieripoll': 2240,
 u'longobardim': 4478,
 u'holaf': 3619,
 u'rossi': 6563,
 u'peatonal': 5647,
 u'with': 7824,
 u'goleador': 3375,
 u'inexplic': 3833,
 u'nort': 5271,
 u'oto\xf1': 5456,
 u'norm': 5268,
 u'sant': 6667,
 u'sanz': 6681,
 u'hast': 3543,
 u'mors': 5029,
 u'silvi': 6850,
 u'mori': 5023,
 u'morb': 5019,
 u'morf': 5022,
 u'mord': 5020,
 u'bendig': 836,
 u'bendit'

In [22]:
tfidf = TfidfTransformer()

In [23]:
tfidf.fit(counts)

TfidfTransformer(norm=u'l2', smooth_idf=True, sublinear_tf=False,
         use_idf=True)

In [24]:
import pickle
with open('tfidf.pickle', 'wb') as f:
    pickle.dump(tfidf, f)

with open('vec.pickle', 'wb') as f:
    pickle.dump(vec, f)

# Load TFIDF model

In [7]:
import pickle
with open('tfidf.pickle', 'rb') as f:
    tfidf = pickle.load(f)

with open('vec.pickle', 'rb') as f:
    vec = pickle.load(f)

## explore tfidf features

In [35]:
text = neg_train_tweets[0].text

In [36]:
tvec = tfidf.transform(vec.transform([text]))

In [66]:
type(tvec)

scipy.sparse.csr.csr_matrix

In [38]:
def tfidf_feats(text):
    tvec = tfidf.transform(vec.transform([text]))

    ivocab = {i: w for (w,i) in vec.vocabulary_.items()}

    return dict(zip([ivocab[i] for i in tvec.indices], tfidf.idf_[tvec.indices]))

tfidf_feats(neg_train_tweets[0].text)


{u'dond': 5.8105590157083755,
 u'eleccion': 5.7517419064040061,
 u'fren': 8.3750991241361668,
 u'juec': 7.7479099113680183,
 u'pasaj': 8.7673540005075239,
 u'quier': 4.903231791074429,
 u'regal': 6.8925279325808564,
 u'sep': 7.5089824923525468,
 u'tucum': 6.9834997107865826,
 u'tvrok': 9.2048218397018626,
 u'vam': 5.7779408633050124}

# Generate TFIDF features

In [6]:
X_train_f = tfidf.transform(vec.transform([t.text for t in train_tweets]))

In [7]:
X_test_f = tfidf.transform(vec.transform([t.text for t in test_tweets]))

In [8]:
X_train_f.shape

(80434, 8067)

In [9]:
X_test_f.shape

(34472, 8067)

In [10]:
fname = join(DATASETS_FOLDER, "tfidf_%d.pickle" % uid)
dataset = (X_train, X_test, y_train, y_test)
pickle.dump(dataset, open(fname, 'wb'))

In [ ]:
dataset = pickle.load(open(fname, 'rb'))
X_train, X_test, y_train, y_test = dataset

## normalize features

In [22]:
type(X_train_f)

scipy.sparse.csr.csr_matrix

In [13]:
from sklearn.preprocessing import StandardScaler

In [11]:
from scipy.sparse import vstack



def _calculate_mean_and_std_deviation(X):
    """
    Calculates mean and standard deviation of sample features.

    Parameters
    ----------
    X : array-like, samples, shape = (n_samples, n_features)
    """

    n_samples, n_features = X.shape

    sigma_ = np.zeros((n_features))
    epsilon = 1e-9

    theta_ = X.mean(axis=0)
    
    for i in range(n_features):
        col = X.getcol(i)

        sqr = col.copy() # take a copy of the col
        sqr.data **= 2 # square the data, i.e. just the non-zero data
        variance = sqr.sum()/n_samples - col.mean()**2

        sigma_[i] = variance + epsilon

    return theta_, sigma_

def _normalize(X):
    """
    Normalizes sample features.

    self.theta_ and self.sigma_ have to be set.

    Parameters
    ----------
    X : array-like, samples, shape = (n_samples, n_features)
    """

    n_samples, n_features = X.shape

    theta_, sigma_ = _calculate_mean_and_std_deviation(X)   

    new_X = np.zeros(shape=(n_samples, n_features), dtype=np.float32)

    new_X[:, :] = (X[:, :] - theta_[:]) / sigma_[:]

    return new_X

def normalize(X_train, X_test):
    train_size = X_train.shape[0]
    X = vstack((X_train, X_test))
    X = _normalize(X)

    X_train = X[:train_size,:]
    X_test = X[train_size:,:]

    return X_train, X_test

def scale(X_train, X_test):
    train_size = X_train.shape[0]
    X = vstack((X_train, X_test))
    X = StandardScaler(with_mean=False).fit_transform(X)
    X_train = X[:train_size,:]
    X_test = X[train_size:,:]

    return X_train, X_test

In [14]:
# X_train, X_test = normalize(X_train_f, X_test_f)
X_train_s, X_test_s = scale(X_train_f, X_test_f)

## class balancing sample weights

In [15]:
# weights for class balancing
w1 = sum(y_train)/len(y_train)
w0 = 1 - w1
sample_weights = np.array([w0 if x==0 else w1 for x in y_train])


In [16]:
from sklearn.svm import SVC

In [17]:
svc=SVC()

## sin normalizar

In [22]:
svc.fit(X_train_f, y_train, sample_weight=sample_weights)
evaluate_model(svc, X_train_f, X_test_f, y_train, y_test)

Detailed classification report:

Scores on training set.

             precision    recall  f1-score   support

        0.0       0.98      1.00      0.99     79167
        1.0       0.00      0.00      0.00      1267

avg / total       0.97      0.98      0.98     80434

Scores on test set.

             precision    recall  f1-score   support

        0.0       0.98      1.00      0.99     33942
        1.0       0.00      0.00      0.00       530

avg / total       0.97      0.98      0.98     34472



## normalizado

In [18]:
svc=SVC()
svc.fit(X_train_s, y_train, sample_weight=sample_weights)
evaluate_model(svc, X_train_s, X_test_s, y_train, y_test)

Detailed classification report:

Scores on training set.

             precision    recall  f1-score   support

        0.0       0.98      1.00      0.99     79167
        1.0       0.00      0.00      0.00      1267

avg / total       0.97      0.98      0.98     80434

Scores on test set.

             precision    recall  f1-score   support

        0.0       0.98      1.00      0.99     33942
        1.0       0.00      0.00      0.00       530

avg / total       0.97      0.98      0.98     34472



## sin normalizar

In [24]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()     
clf.fit(X_train_f, y_train, sample_weight=sample_weights)
evaluate_model(clf, X_train_f, X_test_f, y_train, y_test)

Detailed classification report:

Scores on training set.

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00     79167
        1.0       1.00      0.96      0.98      1267

avg / total       1.00      1.00      1.00     80434

Scores on test set.

             precision    recall  f1-score   support

        0.0       0.99      0.99      0.99     33942
        1.0       0.21      0.14      0.17       530

avg / total       0.97      0.98      0.98     34472



## normalizado

In [19]:
clf = DecisionTreeClassifier()     
clf.fit(X_train_s, y_train, sample_weight=sample_weights)
evaluate_model(clf, X_train_s, X_test_s, y_train, y_test)

Detailed classification report:

Scores on training set.

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00     79167
        1.0       1.00      0.96      0.98      1267

avg / total       1.00      1.00      1.00     80434

Scores on test set.

             precision    recall  f1-score   support

        0.0       0.99      0.99      0.99     33942
        1.0       0.21      0.13      0.16       530

avg / total       0.97      0.98      0.98     34472



## sin normalizar

In [72]:
clf = RandomForestClassifier(n_jobs=-1, n_estimators=20)     
clf.fit(X_train_f, y_train, sample_weight=sample_weights)
evaluate_model(clf, X_train_f, X_test_f, y_train, y_test)

clf = RandomForestClassifier(n_jobs=-1, n_estimators=20)     
clf.fit(X_train, y_train, sample_weight=sample_weights)
evaluate_model(clf, X_train, X_test, y_train, y_test)

Detailed classification report:

Scores on training set.

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00     83716
        1.0       1.00      0.81      0.90      1393

avg / total       1.00      1.00      1.00     85109

Scores on test set.

             precision    recall  f1-score   support

        0.0       0.99      1.00      0.99     35902
        1.0       0.53      0.11      0.18       574

avg / total       0.98      0.98      0.98     36476

Detailed classification report:

Scores on training set.

             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00     83716
        1.0       1.00      0.62      0.77      1393

avg / total       0.99      0.99      0.99     85109

Scores on test set.

             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00     35902
        1.0       0.96      0.64      0.77       574

avg / total       0.99      0.99    

## normalizado

In [20]:
clf = RandomForestClassifier(n_jobs=-1, n_estimators=20)     
clf.fit(X_train_s, y_train, sample_weight=sample_weights)
evaluate_model(clf, X_train_s, X_test_s, y_train, y_test)

Detailed classification report:

Scores on training set.

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00     79167
        1.0       1.00      0.84      0.91      1267

avg / total       1.00      1.00      1.00     80434

Scores on test set.

             precision    recall  f1-score   support

        0.0       0.99      1.00      0.99     33942
        1.0       0.36      0.06      0.10       530

avg / total       0.98      0.98      0.98     34472



In [27]:
import scipy.sparse as sp

X_train_combined = sp.hstack((X_train,X_train_f), format='csr')

In [29]:
X_test_combined = sp.hstack((X_test,X_test_f), format='csr')

In [75]:
clf = RandomForestClassifier(n_jobs=-1, n_estimators=50)     
clf.fit(X_train_combined, y_train, sample_weight=sample_weights)
evaluate_model(clf, X_train_combined, X_test_combined, y_train, y_test)

Detailed classification report:

Scores on training set.

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00     83716
        1.0       1.00      0.96      0.98      1393

avg / total       1.00      1.00      1.00     85109

Scores on test set.

             precision    recall  f1-score   support

        0.0       0.99      1.00      0.99     35902
        1.0       0.52      0.17      0.26       574

avg / total       0.98      0.98      0.98     36476



In [30]:
clf = DecisionTreeClassifier()     
clf.fit(X_train_combined, y_train, sample_weight=sample_weights)
evaluate_model(clf, X_train_combined, X_test_combined, y_train, y_test)

Detailed classification report:

Scores on training set.

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00     79167
        1.0       1.00      0.99      0.99      1267

avg / total       1.00      1.00      1.00     80434

Scores on test set.

             precision    recall  f1-score   support

        0.0       0.99      0.99      0.99     33942
        1.0       0.26      0.17      0.21       530

avg / total       0.98      0.98      0.98     34472



Muy mala calidad en caso NLP puro. Quizás mejore con algo de topic modelling.

Muchísimo overfitting en el caso combinado.

In [44]:
from sklearn.linear_model import LogisticRegression

In [76]:
clf2 = LogisticRegression(penalty='l1', n_jobs=-1, C=500.0)     
clf2.fit(X_train_f, y_train, sample_weight=sample_weights)

evaluate_model(clf2, X_train_f, X_test_f, y_train, y_test)


Detailed classification report:

Scores on training set.

             precision    recall  f1-score   support

        0.0       0.99      1.00      0.99     83716
        1.0       1.00      0.10      0.19      1393

avg / total       0.99      0.99      0.98     85109

Scores on test set.

             precision    recall  f1-score   support

        0.0       0.99      1.00      0.99     35902
        1.0       0.79      0.09      0.16       574

avg / total       0.98      0.99      0.98     36476



In [77]:
clf3 = LogisticRegression(penalty='l1', n_jobs=-1, C=1.0)     
clf3.fit(X_train_f, y_train, sample_weight=sample_weights)

evaluate_model(clf3, X_train_f, X_test_f, y_train, y_test)


Detailed classification report:

Scores on training set.

             precision    recall  f1-score   support

        0.0       0.98      1.00      0.99     83716
        1.0       0.00      0.00      0.00      1393

avg / total       0.97      0.98      0.98     85109

Scores on test set.

             precision    recall  f1-score   support

        0.0       0.98      1.00      0.99     35902
        1.0       0.00      0.00      0.00       574

avg / total       0.97      0.98      0.98     36476



In [90]:
clf4 = LogisticRegression(penalty='l2', class_weight='balanced', n_jobs=-1, C=0.5)     
clf4.fit(X_train_f, y_train)

evaluate_model(clf4, X_train_f, X_test_f, y_train, y_test)


Detailed classification report:

Scores on training set.

             precision    recall  f1-score   support

        0.0       1.00      0.90      0.94     83716
        1.0       0.13      0.96      0.23      1393

avg / total       0.99      0.90      0.93     85109

Scores on test set.

             precision    recall  f1-score   support

        0.0       0.99      0.89      0.94     35902
        1.0       0.08      0.58      0.14       574

avg / total       0.98      0.89      0.93     36476



In [48]:
from sklearn.linear_model import SGDClassifier


In [86]:
clf = SGDClassifier(loss='hinge', penalty='elasticnet', alpha=0.0001,
                    l1_ratio=0.15, fit_intercept=True, n_iter=5,
                    shuffle=True, verbose=0, epsilon=0.1, n_jobs=-1,
                    random_state=None, learning_rate='optimal', eta0=0.0,
                    power_t=0.5, class_weight='balanced', warm_start=False, average=False)

In [87]:
clf.fit(X_train_f, y_train)
evaluate_model(clf, X_train_f, X_test_f, y_train, y_test)

Detailed classification report:

Scores on training set.

             precision    recall  f1-score   support

        0.0       1.00      0.86      0.92     83716
        1.0       0.10      0.94      0.18      1393

avg / total       0.98      0.86      0.91     85109

Scores on test set.

             precision    recall  f1-score   support

        0.0       0.99      0.85      0.92     35902
        1.0       0.06      0.62      0.11       574

avg / total       0.98      0.85      0.90     36476

